In [ ]:
import os
import sys
sys.path.append('../')
from configuration_tiny_mixtral import TinyMixtralConfig
from modeling_tiny_mixtral import TinyMixtralForCausalLM
from src.models.moe.config import ModelConfig
import torch
from dataclasses import asdict
from transformers import AutoTokenizer

In [16]:
def convert_checkpoint_to_hf(
    checkpoint_path,
    output_dir,
    config_dict,
    tokenizer_name="gpt2"
):
    """Convert your checkpoint to HuggingFace format"""
    
    config = TinyMixtralConfig(**config_dict)
    
    hf_model = TinyMixtralForCausalLM(config)
    
    # Load your checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    hf_model.model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Save to HuggingFace format
    hf_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return hf_model, tokenizer

config_dict = asdict(ModelConfig())
config_dict.pop('top_k', None)  # Remove top_k if it exists
# Rename to the HF-expected name
config_dict['top_k_experts'] = ModelConfig.top_k

hf_model, tokenizer = convert_checkpoint_to_hf(
    checkpoint_path="last_epoch_moe_1.pt",
    output_dir="./hf-mixtral",
    config_dict=config_dict
)

/var/folders/16/7k9wy23d4tgf_rq3jzhdr_yc0000gn/T/ipykernel_87663/824331158.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
from huggingface_hub import HfApi, create_repo

def push_to_hub(
    local_dir="./hf-mixtral",
    repo_name="Marmik/tiny-mixtral-5l-active",
    private=False
):
    
    # Create repository
    create_repo(
        repo_id=repo_name,
        private=False,
        exist_ok=True
    )
    
    # Initialize API
    api = HfApi()
    
    # Upload all files
    api.upload_folder(
        folder_path=local_dir,
        repo_id=repo_name,
        repo_type="model"
    )
    
    print(f"Model uploaded to: https://huggingface.co/{repo_name}")

# Push to hub
push_to_hub(
    local_dir="./hf-mixtral",
    repo_name="Marmik/tiny-mixtral-5l-active",
    private=False
)

model.safetensors:   0%|          | 0.00/923M [00:00<?, ?B/s]

Model uploaded to: https://huggingface.co/Marmik/tiny-mixtral-5l-active


In [ ]:
state = torch.load("last_epoch_moe_1.pt", map_location="cpu", weights_only=False)
state_dict = state['model_state_dict']
cfg   = TinyMixtralConfig()
hf_model = TinyMixtralForCausalLM(cfg)
missing, unexpected = hf_model.model.load_state_dict(state_dict, strict=False)
print(missing)
print(unexpected)

In [ ]:
# Generate text using the model
input_ids = torch.tensor([[68, 26, 1024, 38943, 500]])  # Example input tokens
with torch.no_grad():
    output = hf_model.generate(
        input_ids=input_ids,
        max_length=20,
        do_sample=True,
        temperature=0.7,
        pad_token_id=50256
    )
print("Generated output:", output)

In [ ]:
# for name, module in hf_model.named_modules():
#     print(f"{name}: {type(module)}")